# LinkedIn project

### Задача
В текущем проекте займёмся анализом вакансий для джуниоров аналитиков данных, спарсенных из источника LinkedIn




### Цель
Визуализировать информацию о рынке вакансий для аналитиков в Европе

### Описание данных
- job_title - название вакансии
- area - полное название места работы
- city - город места работы
- country - страна места работы
- workplace - тип занятости
- company_name - название компании-работодателя
- ndustry - сфера деятельности компании-работодателя
- company_size - размер компании-работодателя
- job_description - описание вакансии
- posted_date - дата публикации вакансии
- applicants - количество откликов на вакансию.

### План исследования
1. Знакомство с данными.
1. Проверить данные на дубликаты.
2. Обработать пропуски.
3. Удалить не релевантные заданию вакансии.
4. Привести столбец с датой публикации к виду, пригодному для обработки.
5. Выделить hard skills из описания вакансии — их проще искать в разноязычных
     вакансиях, чем soft skills.
6. Удалить ненужные атрибуты (признаки).


### Знакомство с данными 
Загружаем библиотеки и первично ознакомимся с датасетом

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('C:/Users/Admin/Downloads/masterskaya_LinkedIn_2023_05.csv')

In [3]:
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       998 non-null    int64  
 1   job_title        998 non-null    object 
 2   area             998 non-null    object 
 3   city             998 non-null    object 
 4   country          998 non-null    object 
 5   workplace        930 non-null    object 
 6   company_name     996 non-null    object 
 7   industry         964 non-null    object 
 8   company_size     963 non-null    object 
 9   job_description  998 non-null    object 
 10  posted_date      998 non-null    object 
 11  applicants       838 non-null    float64
dtypes: float64(1), int64(1), object(10)
memory usage: 93.7+ KB


,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants
0,0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,47.0
1,1,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job,1 week ago,NaN
2,2,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job\n \n\n \nData Analyst - L...,1 week ago,NaN
3,3,Data Analyst (Space & Planning),"['South Molton', 'England', 'United Kingdom']",South Molton,United Kingdom,On-site,Mole Valley Farmers,NaN,NaN,About the job\n \n\n \nSalary: To be di...,1 week ago,NaN
4,4,Data Analyst,"['Lugano', 'Ticino', 'Switzerland']",Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,About the job\n \n\n \nFORFIRM is provi...,2 weeks ago,NaN
5,5,Data Analyst - Logistics,"['Southampton', 'England', 'United Kingdom']",Southampton,United Kingdom,On-site,"Butler, Bridge & May",NaN,NaN,About the job\n \n\n \nLocation: Southa...,6 days ago,NaN
6,6,Data Analyst,"['Leeds', 'England', 'United Kingdom']",Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,NaN,NaN,About the job\n \n\n \nWe’re Maria Mall...,3 weeks ago,NaN
7,7,Data Analyst,"['Nuneaton', 'England', 'United Kingdom']",Nuneaton,United Kingdom,Hybrid,Kelly Group,NaN,NaN,About the job\n \n\n \nKelly Group are ...,2 days ago,NaN
8,8,Data Analyst,"['Paris', 'Île-de-France', 'France']",Paris,France,On-site,eXalt,NaN,NaN,About the job\n \n\n \nQui sont-ils ?\n...,2 weeks ago,140.0
9,9,Data Analyst - Hybrid Working,"['Cambridge', 'England', 'United Kingdom']",Cambridge,United Kingdom,On-site,Blue Arrow,NaN,NaN,About the job\n \n\n \nData AnalystHybr...,2 weeks ago,NaN


Итак, сданными познакомились. 
- Нам точно не нужен Unnamed: 0, дубль индекса. 
- Судя по info, пропусков не очень много, но поработать с ними можно

### Работа с дубликатами

In [4]:
# проверим колеичество дубликатов в датасете
data.duplicated().sum()

0

In [5]:
dupl =  data[data.duplicated(subset = ['job_title', 'area','workplace','company_name','job_description'], keep='first')]
print(dupl.info())
dupl[['job_title', 'area','workplace','company_name','job_description']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 292 to 831
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       113 non-null    int64  
 1   job_title        113 non-null    object 
 2   area             113 non-null    object 
 3   city             113 non-null    object 
 4   country          113 non-null    object 
 5   workplace        112 non-null    object 
 6   company_name     113 non-null    object 
 7   industry         113 non-null    object 
 8   company_size     113 non-null    object 
 9   job_description  113 non-null    object 
 10  posted_date      113 non-null    object 
 11  applicants       112 non-null    float64
dtypes: float64(1), int64(1), object(10)
memory usage: 11.5+ KB
None


,job_title,area,workplace,company_name,job_description
292,Data Analyst (m/w/d),['Germany'],On-site,Charisma-Tec GmbH,About the job\n \n \n\n\n\n This j...
293,Data Analyst (m/w/d),['Germany'],On-site,Charisma-Tec GmbH,About the job\n \n \n\n\n\n This j...
294,Data Analyst (m/w/d),['Germany'],On-site,Charisma-Tec GmbH,About the job\n \n \n\n\n\n This j...
295,Data Analyst (m/w/d),['Germany'],On-site,Charisma-Tec GmbH,About the job\n \n \n\n\n\n This j...
296,Data Analyst (m/w/d),['Germany'],On-site,Charisma-Tec GmbH,About the job\n \n \n\n\n\n This j...
...,...,...,...,...,...
825,Data Analyste,"['Croix', 'Hauts-de-France', 'France']",On-site,illicado,"About the job\n \n\n \nillicado, N°1 fr..."
826,Data Analyst - Global Marketing H/F,"['Paris', 'Île-de-France', 'France']",On-site,Septodont,About the job\n \n\n \nCompany Details\...
827,BI & data reporting Expert,"['Brussels', 'Brussels Region', 'Belgium']",Hybrid,AXA,About the job\n \n\n \nDe Nederlandse t...
828,Data Engineer,"['Helsinki', 'Uusimaa', 'Finland']",Hybrid,Mandatum,About the job\n \n\n \nMandatum on fina...


In [6]:
data = data[~data.duplicated(subset=['job_title', 'area','workplace','company_name','job_description'], keep='first')]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885 entries, 0 to 997
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       885 non-null    int64  
 1   job_title        885 non-null    object 
 2   area             885 non-null    object 
 3   city             885 non-null    object 
 4   country          885 non-null    object 
 5   workplace        818 non-null    object 
 6   company_name     883 non-null    object 
 7   industry         851 non-null    object 
 8   company_size     850 non-null    object 
 9   job_description  885 non-null    object 
 10  posted_date      885 non-null    object 
 11  applicants       726 non-null    float64
dtypes: float64(1), int64(1), object(10)
memory usage: 89.9+ KB


Удалили 113 неявных дубликатов.

In [7]:
data['country'].unique()

array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany',
       'Brussels Metropolitan Area', 'Portugal',
       'Greater Paris Metropolitan Region', 'Bulgaria', 'Spain',
       'Greater Palma de Mallorca Metropolitan Area', 'Czechia',
       'Warsaw Metropolitan Area', 'Wroclaw Metropolitan Area', 'Latvia',
       'Estonia', 'Greater Barcelona Metropolitan Area', 'Malta',
       'Austria', 'Iasi Metropolitan Area', 'Cologne Bonn Region',
       'Greater Munster Area', 'Greater Banska Bystrica Area',
       'Greater Nuremberg Metropolitan Area', 'Norway',
       'Berlin Metropolitan Area', 'Denmark', 'Finland', 'Romania',
       'Rotterdam and The Hague', 'Prague Metropolitan Area',
       'Greater Pau Area', 'Monaco', 'Lisbon Metropolitan Area',
       'Athens Metropolitan Area', 'Greater Lyon Area',
       'Greater Milan Metropolitan Area', 'Croatia', 'S

Есть "хвост" в виде регионов стран, исправим это вручную.

In [8]:
data['country'] = data['country'].replace({'Greater Paris Metropolitan Region':'France', 'Warsaw Metropolitan Area':'Poland',
                        'Wroclaw Metropolitan Area':'Poland', 'Cologne Bonn Region':'Germany','Greater Munster Area':'Germany',
                        'Greater Banska Bystrica Area':'Slovakia','Greater Nuremberg Metropolitan Area':'Germany',
                        'Berlin Metropolitan Area':'Germany','Greater Barcelona Metropolitan Area':'Spain', 
                         'Rotterdam and The Hague':'Netherlands','Prague Metropolitan Area':'Czechia',
                        'Greater Pau Area':'France', 'Lisbon Metropolitan Area':'Portugal', 'Athens Metropolitan Area':'Greece',
                        'Greater Lyon Area':'France', 'Greater Milan Metropolitan Area':'Italy', 
                        'Stuttgart Region':'Germany', 'Brussels Metropolitan Area':'Belgium',
                        'Greater Oslo Region':'Finland', 'Eindhoven Area':'Netherlands', 
                         'Greater Munich Metropolitan Area':'Germany', 'Krakow Metropolitan Area':'Poland',
                        'Greater Palma de Mallorca Metropolitan Area':'Spain', 'Greater Verona Metropolitan Area':'Italy',
                        'Amsterdam Area':'Netherlands','Copenhagen Metropolitan Area':'Denmark'})
data['country'].unique()

array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany', 'Portugal',
       'Bulgaria', 'Spain', 'Czechia', 'Latvia', 'Estonia', 'Malta',
       'Austria', 'Iasi Metropolitan Area', 'Slovakia', 'Norway',
       'Denmark', 'Finland', 'Romania', 'Monaco', 'Croatia',
       'Greater Dijon Area', 'Greater Madrid Metropolitan Area'],
      dtype=object)

### Работа с пропусками

In [9]:
# проверим количество пропусков в датасете
data.isna().sum()

Unnamed: 0           0
job_title            0
area                 0
city                 0
country              0
workplace           67
company_name         2
industry            34
company_size        35
job_description      0
posted_date          0
applicants         159
dtype: int64

In [10]:
# разберёмся с пропусками в workplace
data['workplace'].unique()

array(['On-site', 'Hybrid', nan, 'Remote'], dtype=object)

Попробуем изучить, может быть есть компании, которые допустили пропуски в формате работы, но прописалаи это в описании вакансии.

In [11]:
data[(data['company_name'].isin(data[data['workplace'].isna()]['company_name'])) 
         & data['job_description'].str.contains('On-site')]

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants


In [31]:
data[(data['company_name'].isin(data[data['workplace'].isna()]['company_name'])) 
     & data['job_description'].str.contains('Hybrid')]

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,junior,date,extracted_skills


In [30]:
data[(data['company_name'].isin(data[data['workplace'].isna()]['company_name'])) 
     & data['job_description'].str.contains('Remote')]

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,junior,date,extracted_skills


Так как пересечений мы не нашли, будем считать, что отсутствие информации по типу занятости означает каноничное присутствие в офисе, что наиболее вероятно.
Также удалим 2 записи с пропусками в компании.

In [13]:
data['workplace'] = data['workplace'].fillna('On-site')

In [14]:
#в company_name у нас всего 2 вакансии, предлагаю удалить эти строки,потери небольшие и мы не можем идентифицировать вакансию
data = data.dropna(subset=['company_name'])

In [15]:
# пропуски в industry
display(data.groupby('industry')['job_title'].count())

industry
Airlines/Aviation                    4
Apparel & Fashion                   13
Automotive                          17
Aviation & Aerospace                 4
Banking                             29
                                    ..
Translation & Localization           3
Transportation/Trucking/Railroad    16
Utilities                           10
Venture Capital & Private Equity     1
Wholesale                            4
Name: job_title, Length: 88, dtype: int64

Здесь сложно принять решение по заполнению, попробуем найти те компании, которые выложили несколько вакансий но где-то допущен пропуск.


In [16]:
data[(data['company_name'].isin(data[data['industry'].isna()]['company_name'])) & ~data['industry'].isna()]

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants
0,0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,47.0
23,23,Data Analyst II,"['Dublin', 'County Dublin', 'Ireland']",Dublin,Ireland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,30.0


Метод неэффективен, признак вторичный, оставляем колонку с пропусками.
Company_size также предлагаю не трогать, признак вторичен.

In [17]:
data[(data['company_name'].isin(data[data['company_size'].isna()]['company_name'])) & ~data['company_size'].isna()]

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants
0,0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,47.0
23,23,Data Analyst II,"['Dublin', 'County Dublin', 'Ireland']",Dublin,Ireland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,30.0


Для company_size метод также неэффективен, признак вторичный, оставляем колонку с пропусками.

In [18]:
# пропуски в applicants
data['applicants'].unique()

array([ 47.,  nan, 140.,  13.,   7.,  53.,  30.,  70., 141.,  36.,  81.,
       125.,  40., 104.,  98.,  19., 110.,  91.,  73., 175.,  56., 190.,
       111., 196.,  88.,  28.,  44.,  99., 136., 138.,  38.,  42., 192.,
        86., 164.,   6., 177.,  66.,  12.,   9., 105., 199., 154.,  84.,
       149.,  72., 116.,  74.,  37.,  79.,  11., 165., 132.,  18., 155.,
       197.,  94.,  54.,  92.,  80.,  33.,  78.,  75.,  20.,  64.,  58.,
        25.,  29., 147., 128.,  14.,  87., 107.,  41.,  26.,  59.,   8.,
        35., 143.,  89.,  48.,  16., 123.,  60., 112., 109.,  82.,  67.,
       113., 180.,  22., 139.,  52.,  55.,  65., 122.,  49., 151., 153.,
       108.,  95.,   4.,  83.,  85., 102.,  51.,  71.,  90.,  68., 168.,
       150.,  96., 178.,  21., 130., 118.,  46., 174.,  62.,  69.,  32.,
       162., 106.,  17.,  27.,   2.,  39.,  34.,  24.,  45., 184.,  63.,
       173., 186.,   5.,  15.,  31.,  23.,   1.,  97.,   3., 146., 115.,
        61., 124., 117., 135., 121., 103., 137.,  9

In [19]:
data['applicants'].median()

47.0

Признак также вторичен, поэтому предлагаю заполнить пропуски медианными значениями, для того, чтобы не сдвигать статистику по выборке.

In [20]:
data['applicants'] = data['applicants'].fillna(47)
data['applicants'].isna().sum()

0

### Удалить не релевантные заданию вакансии

In [21]:
display(data[data['job_title'].str.contains('junior|Junior', regex=True)].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 37 to 996
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       59 non-null     int64  
 1   job_title        59 non-null     object 
 2   area             59 non-null     object 
 3   city             59 non-null     object 
 4   country          59 non-null     object 
 5   workplace        59 non-null     object 
 6   company_name     59 non-null     object 
 7   industry         59 non-null     object 
 8   company_size     59 non-null     object 
 9   job_description  59 non-null     object 
 10  posted_date      59 non-null     object 
 11  applicants       59 non-null     float64
dtypes: float64(1), int64(1), object(10)
memory usage: 6.0+ KB


None

Если рассмотривать прямое упоминание грэйда, то получится очень мало вакансий, поэтому пойдём от обратного, и исключим упоминание более старших грейдов.

In [22]:
# Добавим новый столбец с обозначением, что в названии вакансии есть какие-либо упоминания грэйда
data['junior'] = ~data['job_title'].str.contains('mid|team|med|lead|exp|sen', flags=re.IGNORECASE, regex=True)
data.groupby('junior').count()

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants
junior,,,,,,,,,,,,
False,50,50,50,50,50,50,50,50,50,50,50,50
True,833,833,833,833,833,833,833,801,800,833,833,833


Итак, мы нашли 50 вакансий, в которых прямо упоминается грейд, и на которые лучше не откликаться без опыта.

In [23]:
data = data[data['junior'] == True]
data.shape

(833, 13)

### Работа с датой
Привести столбец с датой публикации к виду, пригодному для обработки

In [24]:
# Функция для преобразования значения "posted_date" в формат времени
def convert_date(posted_date, current_date = datetime.now()):
      # Получаем текущую дату и время

    if "week" in posted_date:
        weeks = int(posted_date.split()[0])
        posted_date = current_date - timedelta(weeks=weeks)
    elif "day" in posted_date:
        days = int(posted_date.split()[0])
        posted_date = current_date - timedelta(days=days)
    else:
        posted_date = current_date

    return posted_date.strftime("%Y-%m-%d")  # Форматируем дату в нужный формат


data['date'] = data['posted_date'].apply(convert_date)
data.head(10)

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,junior,date
0,0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,47.0,True,2023-07-17
1,1,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job,1 week ago,47.0,True,2023-07-17
2,2,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job\n \n\n \nData Analyst - L...,1 week ago,47.0,True,2023-07-17
3,3,Data Analyst (Space & Planning),"['South Molton', 'England', 'United Kingdom']",South Molton,United Kingdom,On-site,Mole Valley Farmers,NaN,NaN,About the job\n \n\n \nSalary: To be di...,1 week ago,47.0,True,2023-07-17
4,4,Data Analyst,"['Lugano', 'Ticino', 'Switzerland']",Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,About the job\n \n\n \nFORFIRM is provi...,2 weeks ago,47.0,True,2023-07-10
5,5,Data Analyst - Logistics,"['Southampton', 'England', 'United Kingdom']",Southampton,United Kingdom,On-site,"Butler, Bridge & May",NaN,NaN,About the job\n \n\n \nLocation: Southa...,6 days ago,47.0,True,2023-07-18
6,6,Data Analyst,"['Leeds', 'England', 'United Kingdom']",Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,NaN,NaN,About the job\n \n\n \nWe’re Maria Mall...,3 weeks ago,47.0,True,2023-07-03
7,7,Data Analyst,"['Nuneaton', 'England', 'United Kingdom']",Nuneaton,United Kingdom,Hybrid,Kelly Group,NaN,NaN,About the job\n \n\n \nKelly Group are ...,2 days ago,47.0,True,2023-07-22
8,8,Data Analyst,"['Paris', 'Île-de-France', 'France']",Paris,France,On-site,eXalt,NaN,NaN,About the job\n \n\n \nQui sont-ils ?\n...,2 weeks ago,140.0,True,2023-07-10
9,9,Data Analyst - Hybrid Working,"['Cambridge', 'England', 'United Kingdom']",Cambridge,United Kingdom,On-site,Blue Arrow,NaN,NaN,About the job\n \n\n \nData AnalystHybr...,2 weeks ago,47.0,True,2023-07-10


### HardSkills
Выделить hard skills из описания вакансии

In [25]:
# Выделим основные скиллы, которые упоминаются в описании вакансий
hard_skills = ['Python', 'SQL', 'Excel','Data Analysis', 'Business Intelligence', 'ETL', 
                   'Modeling','Power BI','Tableau','ML','Linux','HTML','A/B','Statistics','Jupyter']

In [26]:
def extract_hard_skills(df, hard_skills):
    extracted_skills = []
    for index, row in df.iterrows():
        job_description = row['job_description']
        skills = []
        for skill in hard_skills:
            if re.search(skill, job_description, re.IGNORECASE):
                skills.append(skill)
        extracted_skills.append(skills)
    df['extracted_skills'] = extracted_skills
    df_exploded = df.explode('extracted_skills')
    return df_exploded

In [27]:
data_extracted = extract_hard_skills(data, hard_skills)
display(data_extracted.groupby('extracted_skills').count())
data_extracted.info()

,Unnamed: 0,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,junior,date
extracted_skills,,,,,,,,,,,,,,
A/B,28,28,28,28,28,28,28,28,28,28,28,28,28,28
Business Intelligence,139,139,139,139,139,139,139,135,135,139,139,139,139,139
Data Analysis,107,107,107,107,107,107,107,94,94,107,107,107,107,107
ETL,76,76,76,76,76,76,76,75,75,76,76,76,76,76
Excel,401,401,401,401,401,401,401,385,384,401,401,401,401,401
HTML,18,18,18,18,18,18,18,18,18,18,18,18,18,18
Jupyter,11,11,11,11,11,11,11,11,11,11,11,11,11,11
Linux,15,15,15,15,15,15,15,15,15,15,15,15,15,15
ML,78,78,78,78,78,78,78,77,77,78,78,78,78,78


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 997
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2097 non-null   int64  
 1   job_title         2097 non-null   object 
 2   area              2097 non-null   object 
 3   city              2097 non-null   object 
 4   country           2097 non-null   object 
 5   workplace         2097 non-null   object 
 6   company_name      2097 non-null   object 
 7   industry          2025 non-null   object 
 8   company_size      2023 non-null   object 
 9   job_description   2097 non-null   object 
 10  posted_date       2097 non-null   object 
 11  applicants        2097 non-null   float64
 12  junior            2097 non-null   bool   
 13  date              2097 non-null   object 
 14  extracted_skills  1985 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(12)
memory usage: 247.8+ KB


### Очистка ненужных признаков

Перечень ненужных данных:
1. первый столбец, это дубль индекса.
2. area это полное описание, нам потребуется только город и страна
3. job_description мы выделили нужные нам данные из этой графы
4. posted_date дату мы также переработали
5. junior технический столбец, датасет мы уже очистили.

In [28]:
#удалим первый столбец, он дублирует индекс
data_clear = data_extracted.drop(columns=data.columns[[0,2,9,10,12]])
data_clear.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 997
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         2097 non-null   object 
 1   city              2097 non-null   object 
 2   country           2097 non-null   object 
 3   workplace         2097 non-null   object 
 4   company_name      2097 non-null   object 
 5   industry          2025 non-null   object 
 6   company_size      2023 non-null   object 
 7   applicants        2097 non-null   float64
 8   date              2097 non-null   object 
 9   extracted_skills  1985 non-null   object 
dtypes: float64(1), object(9)
memory usage: 180.2+ KB


In [29]:
data_clear.to_csv("LinkedIn_dataset.csv", index=False)

## Вывод
Итак, в ходе нашей обработки данных, мы сделали следующее: 
1. Удалили 133 неявных дубликата в нашем и без того небольшом датасете.
2. Частично обработали пропуски, оставили их только в индустрии и размере компаний.
3. Очистили датасет от явно нерелевантных вакансий.
4. Обработали дату публикации и требования к навыкам.
5. Подготовили датасет к этапу визуализации.
На следующем этапе работы следует обратить внимание на работу сб улевыми значениями выделенных навыков, а также на влияние пропусков в актуальности визуальной части.
